In [1]:
import sys

sys.path.insert(0, "..")
sys.path.insert(0, "../disc_cop")
sys.path.insert(0, "../avg_corr")

import _pickle as pickle
import gymnasium as gym
import numpy as np
import os
import torch

from tqdm.notebook import tqdm

from disc_cop.utils import Buffer
from avg_corr.main import PPOBuffer

from disc_cop.envs import ENV_ID_TO_NAME

In [2]:
datasets_dir = "/Users/chanb/research/ualberta/Avg_OPE/local/orig_train_dataset"
save_datasets_dir = "/Users/chanb/research/ualberta/Avg_OPE/local/datasets/"
split = "train"

In [ ]:
for dataset_path in tqdm(os.listdir(datasets_dir)):
    variant = dataset_path[:-4]

    key_val_pairs = dict()

    key = None
    val = None

    entries = variant.split("-")
    for entry_i, entry in enumerate(entries):
        if (entry_i + 1) % 2 == 0:
            val = entry

            if key == "env":
                val = "-".join(entries[entry_i:])

            assert key not in key_val_pairs
            key_val_pairs[key] = val

        else:
            key = entry

    # print(key_val_pairs)

    complete_path = os.path.join(datasets_dir, dataset_path)
    orig_buffer = pickle.load(open(complete_path, "rb"))
    # print(complete_path)

    buffer_size = int(key_val_pairs["buffer_size"])
    max_len = int(key_val_pairs["max_length"])
    new_buffer = Buffer(
        [orig_buffer.obs_buf.shape[-1]],
        [orig_buffer.act_buf.shape[-1]],
        buffer_size // max_len,
        max_len,
        1,
    )

    for sample_i in range(orig_buffer.max_size):
        obs = orig_buffer.obs_buf[sample_i]
        next_obs = orig_buffer.next_obs_buf[sample_i]
        act = orig_buffer.act_buf[sample_i]
        rew = orig_buffer.rew_buf[sample_i]
        tim = orig_buffer.tim_buf[sample_i]
        logbev = orig_buffer.logbev_buf[sample_i]
        logtarg = orig_buffer.logtarg_buf[sample_i]

        new_buffer.store(
            obs,
            act,
            rew,
            next_obs,
            tim,
            logbev,
            logtarg,
        )

        if tim + 1 == max_len:
            new_buffer.finish_path()

    os.makedirs(
        os.path.join(
            save_datasets_dir,
            "{}-seed_{}".format(
                ENV_ID_TO_NAME[key_val_pairs["env"]],
                key_val_pairs["seed"],
            )
        ),
        exist_ok=True,
    )
    pickle.dump(
        new_buffer,
        open(
            os.path.join(
                save_datasets_dir,
                "{}-seed_{}".format(
                    ENV_ID_TO_NAME[key_val_pairs["env"]],
                    key_val_pairs["seed"],
                ),
                "{}-random_weight_{}-buffer_size_{}-max_len_{}.pkl".format(
                    split,
                    key_val_pairs["random_weight"],
                    key_val_pairs["buffer_size"],
                    key_val_pairs["max_length"],
                )
            ), "wb"
        )
    )
